In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.ops import rnn, rnn_cell
import random

# Hyperparameters

In [2]:
train_length = 1000
test_length = 2000

rnn_size = 10

sampling_rate = 200
striding = 100
train_steps = 10000

In [3]:
train_input_data = []
train_output_data = [] 

In [4]:
input_file_name1 = 'rnn_v6_input_average.csv'
input_file_name2 = 'rnn_v6_input_variation.csv'
input_file_name3 = 'rnn_v6_output.csv'

input_file = open(input_file_name1,'r',newline='')
reader = csv.reader(input_file, delimiter=',')
temp = []
for row in reader:
    temp.append(row)
train_input_data.append(temp)

In [5]:
input_file = open(input_file_name2,'r',newline='')
reader = csv.reader(input_file, delimiter=',')
temp = []
for row in reader:
    temp.append(row)
train_input_data.append(temp)

In [6]:
input_file = open(input_file_name3,'r',newline='')
reader = csv.reader(input_file, delimiter=',')
for row in reader:
    train_output_data.append(row)

In [7]:
train_input_data =np.array(train_input_data, dtype=np.float32)
train_output_data=np.array(train_output_data, dtype=np.float32)
print(train_input_data.shape)
print(train_output_data.shape)

(2, 1000, 134)
(1000, 134)


In [8]:
train_input_data = np.transpose(train_input_data, [2,0,1])
train_output_data = np.transpose(train_output_data,[1,0])
print(train_input_data.shape)
print(train_output_data.shape)

(134, 2, 1000)
(134, 1000)


# Neural network

In [9]:
x = tf.placeholder(tf.float32, shape=[None, 2, train_length])
y = tf.placeholder(tf.float32, shape=[None, train_length])
test_x = tf.placeholder(tf.float32, shape=[None, 2, test_length])

rnn_tei_x = tf.transpose(test_x, [2,0,1])
rnn_tei_x = tf.reshape(rnn_tei_x, [-1,2])
rnn_tei_x = tf.split(0, test_length, rnn_tei_x)

rnn_tri_x = tf.transpose(x, [2,0,1])
rnn_tri_x = tf.reshape(rnn_tri_x, [-1,2])
rnn_tri_x = tf.split(0, train_length, rnn_tri_x)

with tf.variable_scope("rnn") as scope:
    lstm_cell = rnn_cell.LSTMCell(rnn_size)

    W = tf.Variable(tf.truncated_normal([rnn_size,1], stddev=0.01))
    b = tf.Variable(tf.constant(0.01, shape=[1]))
    
    outputs, states = rnn.rnn(lstm_cell, inputs = rnn_tri_x, dtype=tf.float32)
    temp_outputs = tf.pack(outputs)
    temp_outputs = tf.transpose(temp_outputs, [1,0,2])
    temp_outputs = tf.reshape(temp_outputs,[-1,rnn_size])
    temp_outputs = tf.matmul(temp_outputs,W)+b
    temp_outputs = tf.reshape(temp_outputs,[-1,train_length,1])
    print(temp_outputs)
    
    scope.reuse_variables()
    test_outputs, test_states = rnn.rnn(lstm_cell, inputs = rnn_tei_x, dtype =tf.float32)
    temp_test_outputs = tf.pack(test_outputs)
    temp_test_outputs = tf.transpose(temp_test_outputs, [1,0,2])
    temp_test_outputs = tf.reshape(temp_test_outputs,[-1,rnn_size])
    temp_test_outputs = tf.matmul(temp_test_outputs,W)+b
    temp_test_outputs = tf.reshape(temp_test_outputs,[-1,test_length,1])    
    print(temp_test_outputs)
    
    temp_y = tf.reshape(y,[-1,train_length,1])
    print(temp_y)

Tensor("rnn/Reshape_1:0", shape=(?, 1000, 1), dtype=float32)
Tensor("rnn/Reshape_3:0", shape=(?, 2000, 1), dtype=float32)
Tensor("rnn/Reshape_4:0", shape=(?, 1000, 1), dtype=float32)


In [10]:
cost = tf.reduce_mean(tf.square(temp_outputs-temp_y))
train = tf.train.AdamOptimizer(1e-3).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
for step in range(train_steps+1):
    opt, c = sess.run([train, cost], feed_dict={x: train_input_data, y: train_output_data})
    if step%100==0:
        print("{}th run cost = {}".format(step,c))

0th run cost = 0.0654154121875763
100th run cost = 0.04446537420153618
200th run cost = 0.035738877952098846
300th run cost = 0.027245044708251953
400th run cost = 0.02294917032122612
500th run cost = 0.02057117223739624
600th run cost = 0.019496038556098938
700th run cost = 0.018326235935091972
800th run cost = 0.018089210614562035
900th run cost = 0.01657538302242756
1000th run cost = 0.016759853810071945
1100th run cost = 0.017441803589463234
1200th run cost = 0.01588333770632744
1300th run cost = 0.013640363700687885
1400th run cost = 0.01261586882174015
1500th run cost = 0.012009171769022942
1600th run cost = 0.013437283225357533
1700th run cost = 0.011220943182706833
1800th run cost = 0.010744400322437286
1900th run cost = 0.010741069912910461
2000th run cost = 0.010439334437251091
2100th run cost = 0.009985974989831448
2200th run cost = 0.009681904688477516
2300th run cost = 0.009639997035264969
2400th run cost = 0.009181233122944832
2500th run cost = 0.009001169353723526
2600th

In [12]:
tf.add_to_collection('test_input', test_x)
tf.add_to_collection('hypothesis', temp_test_outputs)
saver = tf.train.Saver()
saver.save(sess,'rnn_v6')

'rnn_v6'